# ICAPS 2024 Tutorial: Finding multiple plans - ForbidIterative

ForbidIterative planner can be installed via pip.

In [1]:
from IPython.display import clear_output

!pip install forbiditerative==1.0.13
clear_output(wait=False)

## Domain and Problem Files
We use the iconic problem number one of the gripper domain, where the goal is to move two
balls from room A to room B with a gripper that has two arms.

<br>
<img src="https://github.com/mp-tutorial/mp-tutorial.github.io/blob/main/notebooks/gripper-two-balls.png?raw=true" width=600>
<br>

We need to download the domain and problem files described in PDDL for the planning task we aim to solve. Following this, we read and use these files to create a PDDL problem, setting the quality metric to minimize the plan length.

In [2]:
!wget https://raw.githubusercontent.com/mp-tutorial/mp-tutorial.github.io/main/notebooks/gripper-domain.pddl
!wget https://raw.githubusercontent.com/mp-tutorial/mp-tutorial.github.io/main/notebooks/gripper-prob-two-balls.pddl

def count_plans_by_cost(plans):
    res = {}
    for plan in plans:
        c = plan["cost"]
        if c not in res:
            res[c] = 0
        res[c] += 1
    return res

from forbiditerative import planners
from pathlib import Path

domain_file = Path("gripper-domain.pddl")
problem_file = Path("gripper-prob-two-balls.pddl")

clear_output(wait=False)

## Diverse planning

In [3]:
import json


# Diverse agile planning
plans = planners.plan_diverse_agl(domain_file=domain_file, problem_file=problem_file, number_of_plans_bound=10, timeout=30)
print(count_plans_by_cost(plans["plans"]))

{7: 2, 9: 8}


### Planner output

In [4]:
print(json.dumps(plans["plans"], indent=4))


[
    {
        "cost": 7,
        "actions": [
            "pick ball1 rooma left",
            "move rooma roomb",
            "drop ball1 roomb left",
            "move roomb rooma",
            "pick ball2 rooma left",
            "move rooma roomb",
            "drop ball2 roomb left"
        ]
    },
    {
        "cost": 7,
        "actions": [
            "pick ball1 rooma right",
            "move rooma roomb",
            "drop ball1 roomb right",
            "move roomb rooma",
            "pick ball2 rooma right",
            "move rooma roomb",
            "drop ball2 roomb right"
        ]
    },
    {
        "cost": 9,
        "actions": [
            "pick ball1 rooma left",
            "move rooma roomb",
            "drop ball1 roomb left",
            "move roomb rooma",
            "pick ball2 rooma left",
            "move rooma roomb",
            "pick ball1 roomb right",
            "drop ball1 roomb right",
            "drop ball2 roomb left"
        ]
    },


## Unordered top-quality planning

In [5]:
# FI-unordered-top-quality
plans = planners.plan_unordered_topq(domain_file=domain_file, problem_file=problem_file, quality_bound=1.4, number_of_plans_bound=1000, timeout=30)
print(count_plans_by_cost(plans["plans"]))

{5: 2, 6: 2, 7: 20}


## Subsets top-quality planning 

In [6]:

# FI-subsets-top-quality
plans = planners.plan_subsets_topq(domain_file=domain_file, problem_file=problem_file, quality_bound=1.4, number_of_plans_bound=1000, timeout=30)
print(count_plans_by_cost(plans["plans"]))

{5: 2, 7: 2}
